<a href="https://colab.research.google.com/github/PranavShashidhara/Netflix-Recommendation-system/blob/main/Dictionaries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
!pip install boto3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 80.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.2/83.2 kB 5.5 MB/s eta 0:00:00


In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from sklearn.utils import shuffle
import boto3

In [3]:
data_df = pd.read_parquet('https://netflix-project-data.s3.us-east-1.amazonaws.com/Preprocessed+data/Small_filtered_data.parquet')
data_df.head()

,Movie_id,User_id,Ratings,Date
0,7,2456613,3,2004-03-31
1,7,1979083,2,2004-05-12
2,7,1391523,3,2004-04-27
3,7,1744889,1,2005-08-25
4,7,995497,2,2005-02-14


In [16]:
N = data_df.User_id.max()
data_df.Movie_id.max()
data_df = shuffle(data_df)
cutoff = int(0.8*len(data_df))
df_train = data_df.iloc[:cutoff]
df_test = data_df.iloc[cutoff:]

user2movie = {}
movie2user = {}
usermovie2rating = {}
count = 0
def update_user2movie_and_movie2user(row):
  global count
  count +=1
  if count % 100000 == 0:
    print("preprocessed %.3f" % (float(count)/cutoff))
  #print(row)
  i = int(row.User_id)
  j = int(row.Movie_id)
  if i not in user2movie:
    user2movie[i] = [j]
  else:
    user2movie[i].append(j)

  if j not in movie2user:
    movie2user[j] = [i]
  else:
    movie2user[j].append(i)
  usermovie2rating[(i, j)] = row.Ratings
df_train.apply(update_user2movie_and_movie2user, axis=1)

print("Calling: update_usermovie2rating_test")
count = 0
usermovie2rating_test = {}
def update_usermovie2rating(row):
  global count
  count +=1
  i = int(row.User_id)
  j = int(row.Movie_id)
  usermovie2rating_test[(i, j)] = row.Ratings

df_test.apply(update_usermovie2rating, axis=1)

preprocessed 0.013
preprocessed 0.027
preprocessed 0.040
preprocessed 0.053
preprocessed 0.067
preprocessed 0.080
preprocessed 0.094
preprocessed 0.107
preprocessed 0.120
preprocessed 0.134
preprocessed 0.147
preprocessed 0.160
preprocessed 0.174
preprocessed 0.187
preprocessed 0.200
preprocessed 0.214
preprocessed 0.227
preprocessed 0.240
preprocessed 0.254
preprocessed 0.267
preprocessed 0.281
preprocessed 0.294
preprocessed 0.307
preprocessed 0.321
preprocessed 0.334
preprocessed 0.347
preprocessed 0.361
preprocessed 0.374
preprocessed 0.387
preprocessed 0.401
preprocessed 0.414
preprocessed 0.427
preprocessed 0.441
preprocessed 0.454
preprocessed 0.468
preprocessed 0.481
preprocessed 0.494
preprocessed 0.508
preprocessed 0.521
preprocessed 0.534
preprocessed 0.548
preprocessed 0.561
preprocessed 0.574
preprocessed 0.588
preprocessed 0.601
preprocessed 0.614
preprocessed 0.628
preprocessed 0.641
preprocessed 0.655
preprocessed 0.668
preprocessed 0.681
preprocessed 0.695
preprocessed

,0
6968005,None
3596402,None
8617901,None
8914813,None
2833753,None
...,...
2977941,None
2589705,None
7428316,None
6430478,None


In [20]:
with open('user2movie.json', 'wb') as f:
  pickle.dump(user2movie, f)

with open('movie2user.json', 'wb') as f:
  pickle.dump(movie2user, f)

with open('usermovie2rating.json', 'wb') as f:
  pickle.dump(usermovie2rating, f)

with open('usermovie2rating_test.json', 'wb') as f:
  pickle.dump(usermovie2rating_test, f)